# Topic Embedding

To ground documents in an interpretable space, we map document embeddings to a shared topic space. Topic space reflects the underlying semantic structure of the documents, hence, might provide a more interpretable embedding.

## Inputs

- `models/gpt3/abstracts_gpt3ada.nc` NetCDF4 file containing GPT-3 embeddings.

## Outputs

- `models/gpt3/abstracts_gpt3ada.nc` contains cluster assignments and membership weights of the documents in the topic space.

## Requirements

```bash
mamba activate cogtext
mamba install hdbscan umap-learn # additional packages compared to the NB3 notebook
```

In [1]:
# Setup and imports

%reload_ext autoreload
%reload_ext watermark

%autoreload 2

import xarray as xr

from python.cogtext.topic_model import TopicModel

%watermark
%watermark -iv -p umap,hdbscan,joblib,numpy,numba,pytorch,tensorflow,python.cogtext

Last updated: 2022-05-13T18:56:47.496554+02:00

Python implementation: CPython
Python version       : 3.9.12
IPython version      : 8.3.0

Compiler    : Clang 12.0.1 
OS          : Darwin
Release     : 21.4.0
Machine     : x86_64
Processor   : i386
CPU cores   : 12
Architecture: 64bit

umap          : 0.5.3
hdbscan       : 0.8.28
joblib        : 1.1.0
numpy         : 1.22.3
numba         : 0.53.1
pytorch       : not installed
tensorflow    : 2.7.0
python.cogtext: 0.1.2022051318

xarray: 2022.3.0
sys   : 3.9.12 | packaged by conda-forge | (main, Mar 24 2022, 23:23:20) 
[Clang 12.0.1 ]



First, make sure all the required embeddings and models are available.

In [2]:
# load embeddings dataset
DATASET = xr.load_dataset('models/gpt3/abstracts_gpt3ada.nc')

# load embeddings from the dataset
doc_embeddings = DATASET['gpt3_embeddings'].values
umap_embeddings = DATASET.get('umap_embeddings', None)
DATASET

<xarray.Dataset>
Dimensions:          (pmid: 382855, gpt3_embedding_dim: 1024,
                      original_index: 382855)
Coordinates:
  * pmid             (pmid) int64 34476479 34475632 ... 15533257 11983582
  * original_index   (original_index) int64 0 1 2 3 ... 531740 531745 531746
Dimensions without coordinates: gpt3_embedding_dim
Data variables:
    gpt3_embeddings  (pmid, gpt3_embedding_dim) float64 -0.01878 ... -0.01841

In [3]:
# project document embeddings to a shared topic space

model = TopicModel(parametric_umap=False, verbose=True)
clusters, weights = model.fit_transform(doc_embeddings, umap_embeddings=umap_embeddings)

UMAP(min_dist=0.0, n_components=5, verbose=True)
Fri May 13 18:57:29 2022 Construct fuzzy simplicial set
Fri May 13 18:57:32 2022 Finding Nearest Neighbors
Fri May 13 18:57:32 2022 Building RP forest with 36 trees
Fri May 13 18:58:34 2022 NN descent for 19 iterations
	 1  /  19
	 2  /  19
	 3  /  19
	 4  /  19
	 5  /  19
	 6  /  19
	Stopping threshold met -- exiting after 6 iterations
Fri May 13 18:59:02 2022 Finished Nearest Neighbor Search
Fri May 13 18:59:14 2022 Construct embedding


Epochs completed:   0%|            0/200 [00:00]

Fri May 13 19:04:11 2022 Finished embedding
[TopicModel] Reduced embeddings dimension. Now clustering...
________________________________________________________________________________
[Memory] Calling hdbscan.hdbscan_._hdbscan_boruvka_kdtree...
_hdbscan_boruvka_kdtree(array([[11.132565, ...,  3.557436],
       ...,
       [10.107224, ...,  1.624576]], dtype=float32), 
1, 1.0, 'euclidean', None, 40, True, False, -1)
__________________________________________hdbscan_boruvka_kdtree - 21.7s, 0.4min
[TopicModel] Clustered embeddings. Now computing membership weights...
[TopicModel] Done!


In [6]:
# report
unassigned_documents = (clusters == -1).sum()

print(f'Projected {doc_embeddings.shape[0]} unique documents to a '
      f'{weights.shape[1]}-dimensional topic space, '
      f'while discarding {unassigned_documents} noise documents (unassigned to any of the topics).')

Projected 382855 unique documents to a 515-dimensional topic space, while discarding 172399 noise documents (unassigned to any of the topics).


In [7]:
# store

DATASET['umap_embeddings'] = xr.DataArray(model.umap_embeddings_, dims=['pmid', 'umap_dim'])
DATASET['topics'] = xr.DataArray(clusters, dims=['pmid'])
DATASET['topic_weights'] = xr.DataArray(weights, dims=['pmid', 'topic'])

# documentation
DATASET['umap_embeddings'].attrs['description'] = 'Document embeddinged projected to a 5dim space using UMAP.'
DATASET['topics'].attrs['description'] = 'Assigned topics to each document. -1 indicates unassigned noise documents.'
DATASET['topic_weights'].attrs['description'] = 'membership weights of each document to each topic. The size of the array is N_docs x N_topics.'

# store
DATASET.to_netcdf('models/gpt3/abstracts_gpt3ada.nc',
                  encoding={'gpt3_embeddings':{'zlib': True, 'complevel': 5},
                            'umap_embeddings': {'zlib': True, 'complevel': 5},
                            'topic_weights': {'zlib': True, 'complevel': 5}})

DATASET

<xarray.Dataset>
Dimensions:          (pmid: 382855, gpt3_embedding_dim: 1024,
                      original_index: 382855, umap_dim: 5, topic: 515)
Coordinates:
  * pmid             (pmid) int64 34476479 34475632 ... 15533257 11983582
  * original_index   (original_index) int64 0 1 2 3 ... 531740 531745 531746
Dimensions without coordinates: gpt3_embedding_dim, umap_dim, topic
Data variables:
    gpt3_embeddings  (pmid, gpt3_embedding_dim) float64 -0.01878 ... -0.01841
    umap_embeddings  (pmid, umap_dim) float32 11.13 6.078 1.767 ... 4.498 1.625
    topics           (pmid) int64 -1 331 345 -1 305 -1 ... -1 466 -1 -1 354 -1
    topic_weights    (pmid, topic) float64 0.0001471 0.0004 ... 0.003085